# Bitstamp Assesment Test
## Task 1: Where are my assets?

Welcome to crypto! The Product team is seeking insights into the performance of our lending product. Your mission is to:

1. Clean (if needed) and enrich the attached data (*task_1_earn.csv*) – end goal is having clean table which can be
easily used (without any manipulation) for any kind of analytics. The presented data includes all completed lending
withdrawals (when a user makes a request to stop lending). Add yearweek and yearmonth columns and any other
that will be useful to end users.
Definitions:
    - **User_Id** is the id of user
    - **Id** is the identifier of the withdrawal request
    - **Requested_at** means when the user made a request to unlend
    - **Finished_at** means when the lending provider completed the lending and user got the funds

2. Prepare an analysis of the lending product with the data you have so the product team will be able to identify if we
have any issues with our lending provider. Include numbers and graphs and don't forget to write key findings.
    -  Identify key trends, patterns, and potential issues with the lending provider.

3. Based on your analysis:
    - **Identify opportunities** to improve the **performance** and **reliability** of our lending provider.
    - Suggest actionable **recommendations** for the Product team to address these issues.

##### Import libraries
Import main libraries for analysing data and visualisation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

sns.set_theme(style="whitegrid")


#### Load Data

Import data from the *task_1_earn.csv* file. The tester shoul change the **file_path_1** variable to the path to the file, if it is not already in the same directory.

In [2]:
# Load the CSV file into a DataFrame /w check for file existence
file_path_1 = 'Data/task_1_earn.csv'  # Path to the CSV file (change if the testers file is in a different directory)
try:
    df_1 = pd.read_csv(file_path_1)
    print("Data loaded successfully!")
except FileNotFoundError:
    print(f"File not found at {file_path_1}. Please check the path.")

# Display the first few rows of the DataFrame
df_1.head()


Data loaded successfully!


,currency,user_id,id,amount_native,amount_usd,requested_at,finished_at
0,BTC,44017161,117200,17.449950,1234.825630,2020-11-24 22:59:35,2020-01-25 14:00:02
1,MATIC,46740482,117197,17.450786,546.674743,2020-01-24 22:33:10,2020-01-25 02:00:20
2,PEPE,46489105,117194,17.446612,556.810541,2020-01-24 22:12:18,2020-01-25 02:00:19
3,PEPE,46117080,117193,17.446693,1045.785866,2020-01-24 22:03:20,2020-01-25 02:00:19
4,AVAX,47626266,117191,17.454170,653.661058,2020-01-24 22:01:30,2020-01-25 02:00:41


#### Cleaning the data

Upon inspection we can see that some of the **requested_at** or **finished_at** inputs have invalid values (e. g. requested_at date being February $30^{\text{th}}$). 

We prepare functions that check if the date is valid (*is_valid_date*) and then correct the date (*fix_invalid_date*). The date is corrected to the last valid date before it for values of **requested_at** (e. g. 2020-4-31 is changed to 2020-04-30), and it is corrected to the first valid date after it for values of **finished_at**. All the changes to dates are logged into the data_correction_log.txt file.


In [3]:
# Function to adjust invalid dates
def fix_invalid_date(date_str, adjust='previous'):
    # Split the date and time parts
    date_part, time_part = date_str.split(' ')
    year, month, day = map(int, date_part.split('-'))
    time_str = time_part

    # Adjust the data backwards for requested_at -> parameter 'previous'
    if adjust == 'previous':
        # Move backward until a valid date is found (e.g. 2020-02-31 -> 2020-02-30 -> 2020-02-29)
        while True:
            try:
                new_date = datetime(year, month, day)
                break
            except ValueError:
                day -= 1

    # Adjust the data forward for finished_at -> parameter 'next'
    elif adjust == 'next':
        while True:
            try:
                new_date = datetime(year, month, day)
                break
            except ValueError:
                # Set the date to the first day of the next month
                month += 1
                day = 1

    # Return the fixed date with the original time
    return new_date.strftime('%Y-%m-%d') + ' ' + time_str

# -----------------------------------------------------------------------------------------

# Helper function to parse date flexibly
def is_valid_date(date_str):
    try:
        # Try parsing with fractional seconds
        datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S.%f')
        return True
    except ValueError:
        try:
            # Try parsing without fractional seconds
            datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
            return True
        except ValueError:
            return False

# =========================================================================================

# Initialize a log list to capture changes
log_entries = []

# -----------------------------------------------------------------------------------------

# Process 'requested_at' and 'finished_at' columns
for idx, row in df_1.iterrows():
    # Process 'requested_at'
    original_requested_at = row['requested_at']
    if isinstance(original_requested_at, str):
        if not is_valid_date(original_requested_at):
            new_requested_at = fix_invalid_date(original_requested_at, adjust='previous')
            df_1.at[idx, 'requested_at'] = new_requested_at
            log_entries.append(
                f"Invalid requested_at value for input id: {row['id']}. "
                f"Value changed from {original_requested_at} to {new_requested_at}"
            )

    # Process 'finished_at'
    original_finished_at = row['finished_at']
    if isinstance(original_finished_at, str):
        if not is_valid_date(original_finished_at):
            new_finished_at = fix_invalid_date(original_finished_at, adjust='next')
            df_1.at[idx, 'finished_at'] = new_finished_at
            log_entries.append(
                f"Invalid finished_at value for input id: {row['id']}. "
                f"Value changed from {original_finished_at} to {new_finished_at}"
            )

# -----------------------------------------------------------------------------------------

# Save the log entries to a file
with open('Outputs/data_correction_log.txt', 'w') as log_file:
    for entry in log_entries:
        log_file.write(entry + '\n')

print("Invalid dates fixed. Changes logged to 'date_correction_log.txt'.")


Invalid dates fixed. Changes logged to 'date_correction_log.txt'.


In [4]:
# Convert 'requested_at' and 'finished_at' columns to datetime after all fixes
df_1['requested_at'] = pd.to_datetime(df_1['requested_at'], errors='coerce')
df_1['finished_at'] = pd.to_datetime(df_1['finished_at'], errors='coerce')

# Ensure there are no NaT values after conversion (log and handle if necessary)
if df_1['requested_at'].isnull().any() or df_1['finished_at'].isnull().any():
    print("Warning: Some dates could not be converted to datetime.")

# Add yearweek and yearmonth columns
df_1['yearweek'] = df_1['requested_at'].dt.strftime('%Y-%U')
df_1['yearmonth'] = df_1['requested_at'].dt.strftime('%Y-%m')

# Add time_to_complete column (in hours)
df_1['time_to_complete'] = (df_1['finished_at'] - df_1['requested_at']).dt.total_seconds() / 3600

# Add possible_error column (where time_to_complete is negative)
df_1['possible_error'] = df_1['time_to_complete'] < 0

# Add conversion_rate column (amount_usd / amount_native)
df_1['conversion_rate'] = df_1['amount_usd'] / df_1['amount_native']

# Display the first few rows to verify
print(df_1.head())

# Save the cleaned data to a new CSV file
df_1.to_csv('Outputs/task_1_earn_cleaned.csv', index=False)



  currency   user_id      id  amount_native   amount_usd        requested_at  \
0      BTC  44017161  117200      17.449950  1234.825630 2020-11-24 22:59:35   
1    MATIC  46740482  117197      17.450786   546.674743 2020-01-24 22:33:10   
2     PEPE  46489105  117194      17.446612   556.810541 2020-01-24 22:12:18   
3     PEPE  46117080  117193      17.446693  1045.785866 2020-01-24 22:03:20   
4     AVAX  47626266  117191      17.454170   653.661058 2020-01-24 22:01:30   

          finished_at yearweek yearmonth  time_to_complete  possible_error  \
0 2020-01-25 14:00:02  2020-47   2020-11      -7304.992500            True   
1 2020-01-25 02:00:20  2020-03   2020-01          3.452778           False   
2 2020-01-25 02:00:19  2020-03   2020-01          3.800278           False   
3 2020-01-25 02:00:19  2020-03   2020-01          3.949722           False   
4 2020-01-25 02:00:41  2020-03   2020-01          3.986389           False   

   conversion_rate  
0        70.763847  
1       